# Определение наиболее выгодного региона нефтедобычи

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Описание-проекта" data-toc-modified-id="Описание-проекта-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Описание проекта</a></span></li><li><span><a href="#Загрузка-и-подготовка-данных" data-toc-modified-id="Загрузка-и-подготовка-данных-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Загрузка и подготовка данных</a></span></li><li><span><a href="#Обучение-и-проверка-модели" data-toc-modified-id="Обучение-и-проверка-модели-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Обучение и проверка модели</a></span></li><li><span><a href="#Подготовка-к-расчёту-прибыли" data-toc-modified-id="Подготовка-к-расчёту-прибыли-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Подготовка к расчёту прибыли</a></span></li><li><span><a href="#Расчёт-прибыли-и-рисков" data-toc-modified-id="Расчёт-прибыли-и-рисков-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Расчёт прибыли и рисков</a></span></li></ul></div>

## Описание проекта
От нефтедобывающей компании поступила задача определить, где бурить новую скважину.

Предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. 

Нужно построить модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализировать возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищем месторождения, для каждого определяем значения признаков;
- Строим модель и оцениваем объём запасов;
- Выбираем месторождения с самым высокими оценками значений;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression 
from sklearn.metrics import mean_squared_error

In [2]:
# открываем файлы
try:
    data_0 = pd.read_csv('/datasets/geo_data_0.csv')
    data_1 = pd.read_csv('/datasets/geo_data_1.csv')
    data_2 = pd.read_csv('/datasets/geo_data_2.csv')
except FileNotFoundError: 
    data_0 = pd.read_csv('datasets/geo_data_0.csv')
    data_1 = pd.read_csv('datasets/geo_data_1.csv')
    data_2 = pd.read_csv('datasets/geo_data_2.csv')

In [3]:
data_0

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647
...,...,...,...,...,...
99995,DLsed,0.971957,0.370953,6.075346,110.744026
99996,QKivN,1.392429,-0.382606,1.273912,122.346843
99997,3rnvd,1.029585,0.018787,-1.348308,64.375443
99998,7kl59,0.998163,-0.528582,1.583869,74.040764


In [4]:
data_1

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305
...,...,...,...,...,...
99995,QywKC,9.535637,-6.878139,1.998296,53.906522
99996,ptvty,-10.160631,-12.558096,5.005581,137.945408
99997,09gWa,-7.378891,-3.084104,4.998651,137.945408
99998,rqwUm,0.665714,-6.152593,1.000146,30.132364


In [5]:
data_2

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746
...,...,...,...,...,...
99995,4GxBu,-1.777037,1.125220,6.263374,172.327046
99996,YKFjq,-1.261523,-0.894828,2.524545,138.748846
99997,tKPY3,-1.199934,-2.957637,5.219411,157.080080
99998,nmxp2,-2.419896,2.417221,-5.548444,51.795253


**id** — уникальный идентификатор скважины;

**f0, f1, f2** — три признака точек (неважно, что они означают, но сами признаки значимы);

**product** — объём запасов в скважине (тыс. баррелей).

*Все файлы одинаковые по размеру и названиям столбцов. Оценим пропуски.*

In [6]:
data_0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [7]:
data_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [8]:
data_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [9]:
data_0 = data_0.drop(columns=['id'])
data_1 = data_1.drop(columns=['id'])
data_2 = data_2.drop(columns=['id'])

Пропусков в данных нет, данные уже подготовлены для работы. Мы удалили столбцы с id, они не нужны для обучения модели.

## Обучение и проверка модели

**Обучаем и проверяем модель для каждого региона:**
* Разбиваем данные на обучающую и валидационную выборки в соотношении 75:25.
* Обучаем модель и делаем предсказания на валидационной выборке.
* Сохраняем предсказания и правильные ответы на валидационной выборке.
* Напечатаем на экране средний запас предсказанного сырья и RMSE модели.

In [10]:
predicted = [] # здесь сохраняем предсказания
answers = [] # сохраняем правильные ответы
for data  in [data_0, data_1, data_2]:
    features = data.drop(['product'] , axis=1)
    target = data['product']
    
    
    features_train, features_valid, target_train, target_valid = train_test_split(
        features, target, test_size=0.25, random_state=12345)
    print('Размеры обучающей выборки:', features_train.shape, target_train.shape)
    print('Размеры валидационной выборки:', features_valid.shape, target_valid.shape)
    
    model = LinearRegression()
    model.fit(features_train, target_train)
    predicted_valid = model.predict(features_valid)
    mse = mean_squared_error(target_valid, predicted_valid)
    predicted.append(pd.Series(predicted_valid))
    answers.append(pd.Series(target_valid).reset_index(drop=True))
    
    
    
    print("Linear Regression")
    print('Средний запас предсказанного сырья:', predicted_valid.mean())
    print("MSE =", mse)
    print("RMSE =", mse ** 0.5)
    print()

    predicted_valid = pd.Series(target_train.mean(), index=target_valid.index)
    mse = mean_squared_error(target_valid, predicted_valid)

    print("Mean")
    print("Среднее значение сырья в обучающей выборке:", target_train.mean())
    print("MSE =", mse)
    print("RMSE =", mse ** 0.5)
    print()

Размеры обучающей выборки: (75000, 3) (75000,)
Размеры валидационной выборки: (25000, 3) (25000,)
Linear Regression
Средний запас предсказанного сырья: 92.59256778438035
MSE = 1412.2129364399243
RMSE = 37.5794217150813

Mean
Среднее значение сырья в обучающей выборке: 92.6404677530568
MSE = 1961.5678757223516
RMSE = 44.289591053907365

Размеры обучающей выборки: (75000, 3) (75000,)
Размеры валидационной выборки: (25000, 3) (25000,)
Linear Regression
Средний запас предсказанного сырья: 68.72854689544602
MSE = 0.7976263360391154
RMSE = 0.8930992867756168

Mean
Среднее значение сырья в обучающей выборке: 68.85895465857708
MSE = 2117.9734309299233
RMSE = 46.02144533725471

Размеры обучающей выборки: (75000, 3) (75000,)
Размеры валидационной выборки: (25000, 3) (25000,)
Linear Regression
Средний запас предсказанного сырья: 94.96504596800489
MSE = 1602.3775813236196
RMSE = 40.02970873393434

Mean
Среднее значение сырья в обучающей выборке: 95.03858906371578
MSE = 2016.221007243509
RMSE = 44.

In [11]:
predicted_0 = predicted[0]
predicted_1 = predicted[1]
predicted_2 = predicted[2]

answers_0 = answers[0]
answers_1 = answers[1]
answers_2 = answers[2]

*Метрика RMSE показывает корень из средней квадратичной ошибки, т.е. величину отклонений предсказаний от фактического значения. Наименьшее значение получилось у модели во втором регионе. Это говорит о том, что данные во втором датасете обучили модель наилучшим образом.* 

## Подготовка к расчёту прибыли

**Условия задачи:**
* Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).
* При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
* Бюджет на разработку скважин в регионе — 10 млрд рублей.
* При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
* После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.
* Данные синтетические: детали контрактов и характеристики месторождений не разглашаются.

**Готовимся к расчёту прибыли:**

*  Все ключевые значения для расчётов сохраняем в отдельных переменных.

*  Рассчитываем достаточный объём сырья для безубыточной разработки новой скважины. Сравниваем полученный объём сырья со средним запасом в каждом регионе.


In [12]:
BUDGET = 10**10
OILWELL_NUMBER = 200
OILWELL_INCOME = 450000

In [13]:
product_profit = BUDGET/OILWELL_NUMBER/OILWELL_INCOME
print('Достаточный объём сырья для безубыточной разработки новой скважины:', product_profit)
print('Средний запас предсказанного сырья в первом регионе:', predicted_0.mean())
print('Средний запас предсказанного сырья во втором регионе:', predicted_1.mean())
print('Средний запас предсказанного сырья в третьем регионе:', predicted_2.mean())

Достаточный объём сырья для безубыточной разработки новой скважины: 111.11111111111111
Средний запас предсказанного сырья в первом регионе: 92.59256778438005
Средний запас предсказанного сырья во втором регионе: 68.7285468954458
Средний запас предсказанного сырья в третьем регионе: 94.96504596800504


*Средний запас сырья по предсказаниям ниже требуемого для безубыточной разработки, но мы будем брать только 200 лучших по предсказанию скважин, где объемы должны быть выше среднего.*

## Расчёт прибыли и рисков 

Напишем функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели:

* Выбераем скважины с максимальными значениями предсказаний.

* Просуммируем целевое значение объёма сырья, соответствующее этим предсказаниям.

* Рассчитываем прибыль для полученного объёма сырья.

In [14]:
def profit(target, predicted,OILWELL_NUMBER):
    predict_sorted = predicted.sort_values(ascending=False) # сортируем предсказанные значения сырья
    selected = target[predict_sorted.index][:OILWELL_NUMBER] # выбираем целевое значение, соотвествующее предсказанию по индексу
    
    return (selected.sum()*OILWELL_INCOME-BUDGET)/10**9

In [15]:
print("Прибыль по первому региону {:.3f} млрд рублей".format(profit(answers_0, predicted_0, OILWELL_NUMBER)))
print("Прибыль по второму региону {:.3f} млрд рублей".format(profit(answers_1, predicted_1, OILWELL_NUMBER)))
print("Прибыль по третьему региону {:.3f} млрд рублей".format(profit(answers_2, predicted_2, OILWELL_NUMBER)))

Прибыль по первому региону 3.321 млрд рублей
Прибыль по второму региону 2.415 млрд рублей
Прибыль по третьему региону 2.710 млрд рублей


Посчитаем риски и прибыль для каждого региона:

* Применяем технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли.

* Находим среднюю прибыль, 95%-й доверительный интервал и риск убытков. Убыток — это отрицательная прибыль.


In [16]:
state = np.random.RandomState(12345)
    
values = []

for i in range(1000):
    target_subsample = answers_0.sample(n=500, replace=True, random_state=state)
    predict_subsample = predicted_0[target_subsample.index]
    values.append(profit(target_subsample, predict_subsample, 200))
    
values = pd.Series(values)    

print('Вероятность убытка:', (values < 0).mean()*100, '%')


lower = values.quantile(0.025)
upper = values.quantile(0.975)
mean = values.mean()
print("Средняя выручка:", mean)
print('Доверительный интервал выручки')
print("2.5%-квантиль:", lower)
print("97.5%-квантиль:", upper)

Вероятность убытка: 6.0 %
Средняя выручка: 0.42593852691059225
Доверительный интервал выручки
2.5%-квантиль: -0.10209009483793655
97.5%-квантиль: 0.9479763533583688


In [17]:
state = np.random.RandomState(12345)
    
values = []

for i in range(1000):
    
    target_subsample = answers_1.sample(n=500, replace=True, random_state=state)
    predict_subsample = predicted_1[target_subsample.index]
    values.append(profit(target_subsample, predict_subsample, 200))
  
    
values = pd.Series(values)
print('Вероятность убытка:', (values < 0).mean()*100, '%')


lower = values.quantile(0.025)
upper = values.quantile(0.975)
mean = values.mean()
print("Средняя выручка:", mean)
print('Доверительный интервал выручки')
print("2.5%-квантиль:", lower)
print("97.5%-квантиль:", upper)

Вероятность убытка: 1.0 %
Средняя выручка: 0.5152227734432894
Доверительный интервал выручки
2.5%-квантиль: 0.06887322537050175
97.5%-квантиль: 0.9315475912570494


In [18]:
state = np.random.RandomState(12345)
    
values = []

for i in range(1000):
    target_subsample = answers_2.sample(n=500, replace=True, random_state=state)
    predict_subsample = predicted_2[target_subsample.index]
    values.append(profit(target_subsample, predict_subsample, 200))
    
values = pd.Series(values)
print('Вероятность убытка:', (values < 0).mean()*100, '%')


lower = values.quantile(0.025)
upper = values.quantile(0.975)
mean = values.mean()
print("Средняя выручка:", mean)
print('Доверительный интервал выручки')
print("2.5%-квантиль:", lower)
print("97.5%-квантиль:", upper)

Вероятность убытка: 6.4 %
Средняя выручка: 0.4350083627827557
Доверительный интервал выручки
2.5%-квантиль: -0.128880547329789
97.5%-квантиль: 0.9697069541802662


*Риск убытков меньше 2,5% только во втором регионе. Поэтому для разработки рекомендуется рассматривать второй регион geo_1.*

**Вывод** 

Проведенные исследования показали, что минимальный риск убытков и самая высокая средняя выручка во втором регионе geo_1, поэтому его следует рассматривать для разработки скважин.